# Import Files

In [32]:
import numpy as np
import pandas as pd
import librosa as lb
import tensorflow as tf
import os
import pathlib
from keras.utils import to_categorical
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm

# Import Dataset

In [12]:
if os.path.isdir("audio"):
    audio_dataset = pathlib.Path('audio')
    # Create list of all audio files for tensorflow
    audio_wav = tf.io.gfile.glob(str(audio_dataset) + '/*')
else:
    exit("No audio dataset folder found")

print(audio_wav)

['audio\\BF001_1PT.wav', 'audio\\BF001_2NL.wav', 'audio\\BF001_3NT.wav', 'audio\\BF001_4PL.wav', 'audio\\BF002_1PT.wav', 'audio\\BF002_2NL.wav', 'audio\\BF002_3NT.wav', 'audio\\BF002_4PL.wav', 'audio\\BF003_1PT.wav', 'audio\\BF003_2NL.wav', 'audio\\BF003_3NT.wav', 'audio\\BF003_4PL.wav', 'audio\\BF004_1PT.wav', 'audio\\BF004_2NL.wav', 'audio\\BF004_3NT.wav', 'audio\\BF004_4PL.wav', 'audio\\BF006_1PT.wav', 'audio\\BF006_2NL.wav', 'audio\\BF006_3NT.wav', 'audio\\BF006_4PL.wav', 'audio\\BF008_1PT.wav', 'audio\\BF008_2NL.wav', 'audio\\BF008_3NT.wav', 'audio\\BF008_4PL.wav', 'audio\\BF013_1PT.wav', 'audio\\BF013_2NL.wav', 'audio\\BF013_3NT.wav', 'audio\\BF013_4PL.wav', 'audio\\BF014_1PT.wav', 'audio\\BF014_2NL.wav', 'audio\\BF014_3NT.wav', 'audio\\BF014_4PL.wav', 'audio\\BF015_1PT.wav', 'audio\\BF015_2NL.wav', 'audio\\BF015_3NT.wav', 'audio\\BF015_4PL.wav', 'audio\\BF016_1PT.wav', 'audio\\BF016_2NL.wav', 'audio\\BF016_3NT.wav', 'audio\\BF016_4PL.wav', 'audio\\BF018_1PT.wav', 'audio\\BF018_2

# Function Definitions

In [41]:
def wav2mfcc(file_path, max_pad_len=20):
    wave, sr = lb.core.load(file_path)
    wave = wave[::3]
    mfcc = lb.feature.mfcc(wave, sr=8000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc

def get_data():

    labels = []
    mfccs = []

    for f in os.listdir('./recordings'):
        if f.endswith('.wav'):
            # MFCC
            mfccs.append(wav2mfcc('./recordings/' + f))

            # List of labels
            label = f.split('_')[0]
            labels.append(label)

    return np.asarray(mfccs), to_categorical(labels)



# Audio Processing

In [53]:
mfcc = []

for audio_path in audio_wav:
    mfcc.append(wav2mfcc(audio_path, 600))
    print(audio_path)

plt.imshow(mfcc[0], aspect='auto', cmap=cm.viridis)
plt.show()
print("FINISHED")

audio\BF001_1PT.wav
audio\BF001_2NL.wav
audio\BF001_3NT.wav
audio\BF001_4PL.wav
audio\BF002_1PT.wav
audio\BF002_2NL.wav
audio\BF002_3NT.wav
audio\BF002_4PL.wav
audio\BF003_1PT.wav
audio\BF003_2NL.wav
audio\BF003_3NT.wav
audio\BF003_4PL.wav
audio\BF004_1PT.wav
audio\BF004_2NL.wav
audio\BF004_3NT.wav
audio\BF004_4PL.wav
audio\BF006_1PT.wav
audio\BF006_2NL.wav
audio\BF006_3NT.wav
audio\BF006_4PL.wav
audio\BF008_1PT.wav
audio\BF008_2NL.wav
audio\BF008_3NT.wav
audio\BF008_4PL.wav
audio\BF013_1PT.wav
audio\BF013_2NL.wav


KeyboardInterrupt: 

# Results